## Some background
- The complete code is: the train number (001-120) + the coach number (there are 6 coaches in a train) + the door number (4 doors in a coach). However, the dataset only has train number (column G), so the train number is enough.
- Note: even though the DIN code is present in the dataset and also in the pdfs provided, there is no relationship between them (Iguess the pdfs provided are not related to this specific dataset in general)
- After having analyzed the old dataset, and after all the questions I asked on Teams, the supervisor sent me the new dataset (componentwise). You can discard the old one and work on the componentwise!

### OLD:

In [ ]:
import pandas as pd
datapath = "data/raw/doorwise_dataset.xlsx"

cols_en = [
"Owner Name_2 (TPL)",
"Vehicle family (TPL)",
"EFA No (MEL)",
"DIN code UPG (H)",
"Assembly (DIN code) (MEL)",
"Vehicle type (TPL)",
"Multiple unit / EW (TPL)",
"Functional location (TPL)",
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Material (CO)",
"Material reference (MAT)",
"Completion date (MEL)",
"Number of messages",
"Number of orders",
"Direct manufacturing costs ACTUAL [CHF] ",
"Material quantity ACTUAL",
"Actual expenditure [h]"
]

cols_interst = [
"Multiple unit / EW (TPL)",
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)"
]

In [ ]:
df = pd.read_excel(datapath, sheet_name="Sheet1")
df.columns = cols_en
df = df[cols_interst]

# Column G of the original dataset is the train number. Actually, only the digits after the 94 85 0511.
# Hence I'm going to remove the prefix (first check if the prefix doesn't change, maybe it has a meaning)
df.rename(columns={"Multiple unit / EW (TPL)": "train_number",
                   "Fault start date (MEL)" : "fault_start_date",
                   "Short text (MEL)" : "problem",
                   "Long text (MEL)" : "maintenance_log",
                   "Message number (MEL)" : "log_number"}, inplace=True)

In [ ]:
tmp = None
for elem in df["train_number"]:
    e = elem[:10]
    if tmp == None or tmp != e:
        tmp = e
        print(tmp)

# Ok it is always the same. Remove the prefix:
df["train_number"]= df["train_number"].str[10:]

In [ ]:
df.head()

## New dataset (componentwise)

In [ ]:
import pandas as pd
datapath = "data/raw/componentwise_dataset.xlsx"
df = pd.read_excel(datapath, sheet_name="Export")

cols_en = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"Vehicle family (TPL)",
"Vehicle type (TPL)",
"Material (CO)",
"Material reference (MAT)",
"Material quantity ACTUAL",
"Material / unit of measure (CO)",
"WF screen symptom (MEL)",
"WF-cause-sys-1 (MEL) (H)",
"WF-cause-sys-2 (MEL) (H)",
"WF-Cause-Sys-3 (MEL) (H)",
"WF-Cause-Sys-4 (MEL) (H)",
"WF-Cause-Sys-5 (MEL) (H)",
"WF-cause-sys-6 (MEL) (H)",
"WF cause failure type (MEL)",
"WF-cause-fix.mass (1-3) (MEL)",
"Multiple unit / EW (TPL)",
"Functional location (TPL)",
"DIN code (H)",
"Design workstation (location) (MEL)"
]

cols_interst = [
"Fault start date (MEL)",
"Short text (MEL)",
"Long text (MEL)",
"Message number (MEL)",
"WF-cause-sys-1 (MEL) (H)",
"WF-cause-sys-2 (MEL) (H)",
"WF-Cause-Sys-3 (MEL) (H)",
"WF-Cause-Sys-4 (MEL) (H)",
"Multiple unit / EW (TPL)"
]


In [ ]:
# At the end there are some unnecessary inhomogenous rows. I filter out them
# Filter out rows from index 1595 (inclusive) onwards
df = df.loc[:1594]

In [ ]:
for i, col in enumerate(df.columns):
    print(i, col)

In [ ]:
df.columns = cols_en
df = df[cols_interst]

# Column G of the original dataset is the train number. Actually, only the digits after the 94 85 0511.
# Hence I'm going to remove the prefix (first check if the prefix doesn't change, maybe it has a meaning)
df.rename(columns={"Multiple unit / EW (TPL)": "train_number",
                   "Fault start date (MEL)" : "fault_start_date",
                   "Short text (MEL)" : "problem",
                   "Long text (MEL)" : "maintenance_log",
                   "Message number (MEL)" : "log_number",
                   "WF-cause-sys-1 (MEL) (H)" : "cause_sys1",
                   "WF-cause-sys-2 (MEL) (H)" : "cause_sys2",
                   "WF-Cause-Sys-3 (MEL) (H)" : "cause_sys3",
                   "WF-Cause-Sys-4 (MEL) (H)" : "cause_sys4"}, inplace=True)

In [ ]:
# Check datatypes and convert them
print(df.dtypes)
print()

df["fault_start_date"] = pd.to_datetime(df["fault_start_date"], format="%Y-%m-%d %H:%M:%S")
df["probelm"] = df["problem"].astype(str)
df["maintenance_log"] = df["maintenance_log"].astype(str)
df["log_number"] = df["log_number"].astype(int)
df["cause_sys1"] = df["cause_sys1"].astype(str)
df["cause_sys2"] = df["cause_sys2"].astype(str)
df["cause_sys3"] = df["cause_sys3"].astype(str)
df["cause_sys4"] = df["cause_sys4"].astype(str)
df["train_number"] = df["train_number"].astype(str)


print(df.dtypes)

In [ ]:
tmp = None
for elem in df["train_number"]:
    e = str(elem)[:10]
    if tmp == None or tmp != e:
        tmp = e
        print(tmp)

# Ok it is always the same. Remove the prefix:
df["train_number"]= df["train_number"].str[10:]

# Reorder columns
df = df[["train_number", "problem", "log_number", "maintenance_log", "fault_start_date", 
         "cause_sys1", "cause_sys2", "cause_sys3", "cause_sys4"]]

In [ ]:
df.head()

In [ ]:
# Save interim dataframe
df.to_csv("data/interim/componentwise_df.csv", index=False)

## Component xlsx

In [ ]:
# There are 2 sheets. I don't know the difference. Need to ask to supervisor
import pandas as pd
datapath = "data/raw/component.xlsx"
df1 = pd.read_excel(datapath, sheet_name="BBA Nummern im WmP-Baum")
df2 = pd.read_excel(datapath, sheet_name="V10")

# Set proper column names
df1.columns = df1.iloc[0]
df1 = df1[1:]
df1.reset_index(drop=True, inplace=True)

df2.columns = df2.iloc[0]
df2 = df2[1:]
df2.reset_index(drop=True, inplace=True)

In [ ]:
# Non ho idea se la colonna Ebene (level) serva. 
# Spero di cuore che la colonna Code ci consenta di matchare i codici delle parti su Unity
cols_interst = [
"Code",
"Ebene",
"DIN-Code",
"Bezeichnung Italienisch\nmax. 60 Char"
]

df1 = df1[cols_interst]
df2 = df2[cols_interst]

# Remove last 2 rows from df1 since they are unnecessary
df1 = df1.loc[:2302]

In [ ]:
df1.rename(columns={"Code": "component_code",
                   "Ebene" : "level",
                   "DIN-Code" : "DIN_code",
                   "Bezeichnung Italienisch\nmax. 60 Char" : "component_name"}, inplace=True)

df2.rename(columns={"Code": "component_code",
                   "Ebene" : "level",
                   "DIN-Code" : "DIN_code",
                   "Bezeichnung Italienisch\nmax. 60 Char" : "component_name"}, inplace=True)

In [ ]:
# To remove the DIN_Code issue with NaN that is recognized ass part of the other DIN_Code column
df1.columns = [f"{col}_{i}" if df1.columns.tolist().count(col) > 1 else col for i, col in enumerate(df1.columns)]
df2.columns = [f"{col}_{i}" if df2.columns.tolist().count(col) > 1 else col for i, col in enumerate(df2.columns)]

df1.drop("DIN_code_3", axis=1, inplace=True)
df2.drop("DIN_code_3", axis=1, inplace=True)

df1.rename(columns={"DIN_code_2" : "DIN_code"}, inplace=True)
df2.rename(columns={"DIN_code_2" : "DIN_code"}, inplace=True)


In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df1.tail()

In [ ]:
df2.tail()

In [ ]:
# Print the differences between the 2 dataframes
differences = df1.compare(df2)

print("Differences between the DataFrames:")
print(differences)

In [ ]:
# I save both
df1.to_csv("data/interim/components1.csv", index=False)
df2.to_csv("data/interim/components2.csv", index=False)

## Data Processing

In [65]:
# Imports
import re
import pandas as pd

# Config
datapath = "data/interim/componentwise_df.csv"
component_cols = ["cause_sys1","cause_sys2", "cause_sys3", "cause_sys4"]

# Utils
def extract_door_number(text):
    pattern = r'(?:Localisation \(Cause\)|Lokalisierung \(Störungsursache\)):\s*\*?\s*(?:Türe|Porte)\s+(\d+)\s*\*?'
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return None

In [66]:
df = pd.read_csv(datapath)
df.head()

,train_number,problem,log_number,maintenance_log,fault_start_date,cause_sys1,cause_sys2,cause_sys3,cause_sys4
0,004-9,+ET-Notöffnug-Scheibe 1 fehlt,131511699,* 26.09.2024 15:07:37 CET (U241403) * System ...,2024-09-26,090 - Einstiegssysteme,010 - Einstiegstüre,050 - Türnotöffnung,010 - Einschlagscheibe Türnotöffnung
1,026-2,ET vitre ouv.secours 2 cassé,131506992,* 25.09.2024 12:01:18 CET (U244539) * Système...,2024-09-25,090 - Einstiegssysteme,010 - Einstiegstüre,050 - Türnotöffnung,010 - Einschlagscheibe Türnotöffnung
2,058-5,Sch.Tritt Zahnriemen 1 abgenutzt,131503871,* 25.09.2024 00:07:35 CET (U244162) * System ...,2024-09-25,090 - Einstiegssysteme,030 - Schiebetritt,060 - Trittmechanik,040 - Spindel / Zahnriemen
3,004-9,-ET 3 schliesst nicht,131500313,* 24.09.2024 06:00:37 CET (U222454) Tel. +41 ...,2024-09-24,090 - Einstiegssysteme,010 - Einstiegstüre,NaN,NaN
4,018-9,-ET calculateur porte 4 bloqué,131489381,* 20.09.2024 22:58:01 CET (U207064) Tél. +41 ...,2024-09-20,090 - Einstiegssysteme,010 - Einstiegstüre,020 - Türsteuerung lokal,010 - Türrechner


In [67]:
# Separate (potential) component code from component name and create dedicated column
for i, colname in enumerate(component_cols):
    df[f"comp_cause_code_{i+1}"] = df[colname].str.split(" - ").str[0]
    df[f"comp_cause_name_{i+1}"] = df[colname].str.split(" - ").str[1]

In [68]:
# Quick check for NaN values across all component columns
if df[component_cols].isna().all().any():
    print("There are NaN values for all columns in the subset.")
else:
    print("There are no NaN values for all columns in the subset.")

There are no NaN values for all columns in the subset.


In [69]:
# Drop the original columns as no longer needed
df.drop(component_cols, axis=1, inplace=True)

In [70]:
# Look for patterns to find door number and create dedicated column
df["door"] = df["maintenance_log"].apply(extract_door_number)

In [71]:
# Check which rows have NaN or None values
# ==> these rows have nothing to do with the external door. They are related to other "systems"
subset = df[df["door"].isna() | (df["door"].isnull())]
#subset.head()

df = df.dropna(subset=["door"])
df.reset_index(drop=True, inplace=True)

In [72]:
# Final df (only door rows, train_id + door_id, component_id + name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 904 entries, 0 to 903
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   train_number       904 non-null    object
 1   problem            904 non-null    object
 2   log_number         904 non-null    int64 
 3   maintenance_log    904 non-null    object
 4   fault_start_date   904 non-null    object
 5   comp_cause_code_1  904 non-null    object
 6   comp_cause_name_1  904 non-null    object
 7   comp_cause_code_2  904 non-null    object
 8   comp_cause_name_2  904 non-null    object
 9   comp_cause_code_3  817 non-null    object
 10  comp_cause_name_3  817 non-null    object
 11  comp_cause_code_4  729 non-null    object
 12  comp_cause_name_4  729 non-null    object
 13  door               904 non-null    object
dtypes: int64(1), object(13)
memory usage: 99.0+ KB


In [73]:
df.head(10)

,train_number,problem,log_number,maintenance_log,fault_start_date,comp_cause_code_1,comp_cause_name_1,comp_cause_code_2,comp_cause_name_2,comp_cause_code_3,comp_cause_name_3,comp_cause_code_4,comp_cause_name_4,door
0,004-9,+ET-Notöffnug-Scheibe 1 fehlt,131511699,* 26.09.2024 15:07:37 CET (U241403) * System ...,2024-09-26,090,Einstiegssysteme,010,Einstiegstüre,050,Türnotöffnung,010,Einschlagscheibe Türnotöffnung,1
1,026-2,ET vitre ouv.secours 2 cassé,131506992,* 25.09.2024 12:01:18 CET (U244539) * Système...,2024-09-25,090,Einstiegssysteme,010,Einstiegstüre,050,Türnotöffnung,010,Einschlagscheibe Türnotöffnung,2
2,058-5,Sch.Tritt Zahnriemen 1 abgenutzt,131503871,* 25.09.2024 00:07:35 CET (U244162) * System ...,2024-09-25,090,Einstiegssysteme,030,Schiebetritt,060,Trittmechanik,040,Spindel / Zahnriemen,1
3,004-9,-ET 3 schliesst nicht,131500313,* 24.09.2024 06:00:37 CET (U222454) Tel. +41 ...,2024-09-24,090,Einstiegssysteme,010,Einstiegstüre,NaN,NaN,NaN,NaN,3
4,018-9,-ET calculateur porte 4 bloqué,131489381,* 20.09.2024 22:58:01 CET (U207064) Tél. +41 ...,2024-09-20,090,Einstiegssysteme,010,Einstiegstüre,020,Türsteuerung lokal,010,Türrechner,4
5,052-8,Schiebetritt Führung 3 schwergängig,131483499,* 19.09.2024 13:57:52 CET (U237164) * System (...,2024-09-19,090,Einstiegssysteme,030,Schiebetritt,060,Trittmechanik,030,Führungselement,3
6,053-6,+ET-Taster öffnen aus 2 leuchtet ni...,131482054,* 19.09.2024 10:17:14 CET (U208328) Tel. +41 ...,2024-09-19,090,Einstiegssysteme,010,Einstiegstüre,030,Bedienelemente Türe,005,Türöffnung,2
7,053-6,+ET-Taster öffnen aus 2 leuchtet ni...,131482054,* 19.09.2024 10:17:14 CET (U208328) Tel. +41 ...,2024-09-19,090,Einstiegssysteme,010,Einstiegstüre,030,Bedienelemente Türe,005,Türöffnung,2
8,015-5,Primärdrehfalle tauschen,131479425,* 18.09.2024 12:56:19 CET (U239359) * System ...,2024-09-18,090,Einstiegssysteme,010,Einstiegstüre,090,Türmechanik,030,Ver-/ Entriegelungseinheit,3
9,001-5,-ET-Antriebsmotor 1 gestört-zeitwei...,131471966,* 17.09.2024 09:18:21 CET (U147283) Tel. +41 ...,2024-09-17,090,Einstiegssysteme,010,Einstiegstüre,080,Türantrieb elektr.,010,Motor zu Türantrieb,1
